
# BĐS Đà Nẵng → CSV 


## 0) Cài thư viện
Cài xong thư viện thì nhấn restart rồi từ lần sau không cần chạy cell này nữa

In [ ]:
%pip install -U pip setuptools wheel
%pip install undetected-chromedriver selenium beautifulsoup4 lxml pandas

## 1) Cấu hình
- BASE_URLS muốn lấy thêm nhiều chỗ khác thì sửa hoặc thêm chỗ này
- START_PAGE: Bắt đầu từ trang nào
- N_PAGES: muốn nó lấy bao nhiêu trang 
> Còn lại không phải chỉnh gì

In [ ]:
import re, time, random
from pathlib import Path
from typing import List, Dict, Optional
import pandas as pd
from bs4 import BeautifulSoup

BASE_URLS = {
    "Tổng hợp Đà Nẵng": "https://batdongsan.com.vn/nha-dat-ban-da-nang",
    "Bán căn hộ chung cư": "https://batdongsan.com.vn/ban-can-ho-chung-cu-da-nang",
    "Bán nhà riêng": "https://batdongsan.com.vn/ban-nha-rieng-da-nang",
    "Bán nhà mặt phố": "https://batdongsan.com.vn/ban-nha-mat-pho-da-nang",
    "Bán đất": "https://batdongsan.com.vn/ban-dat-da-nang",
    
}
BASE_URL   = BASE_URLS["Bán nhà riêng"]
START_PAGE = 2
N_PAGES    = 1
OUTPUT_CSV = "bds.csv"
APPEND_MODE = True
DELAY_RANGE = (1.5, 2.5)
HEADLESS = False  # khuyến nghị để giảm chặn; bật True nếu cần chạy nền
VERBOSE_DETAIL = False  # tắt/mở log chi tiết

COLS9 = ["ngay_dang","loai_hinh","dien_tich","gia","giay_to_phap_ly","so_phong_ngu","so_phong_ve_sinh","tinh_trang_noi_that","link"]

def text_clean(x: Optional[str]) -> str:
    return re.sub(r"\s+", " ", x or "").strip()

def pause(a: Optional[float] = None, b: Optional[float] = None) -> None:
    """Ngủ ngẫu nhiên trong khoảng [a,b]; nếu không truyền thì dùng DELAY_RANGE."""
    if a is None or b is None:
        a, b = DELAY_RANGE
    time.sleep(random.uniform(a, b))

## 2) Lấy link ở danh sách (Selenium headful)

In [ ]:

def parse_list_page(html: str):
    soup = BeautifulSoup(html, "lxml")
    cards = soup.select("div.re__card, div.property-item, div.js__card")
    out = []
    if cards:
        for c in cards:
            a = c.select_one("a[href*='/ban-'], a[href*='/nha-dat-'], a[href*='/can-ho-'], a[href*='/dat-']")
            if not a:
                continue
            href = a.get("href","")
            link = href if href.startswith("http") else ("https://batdongsan.com.vn"+href)
            out.append({"link": link})
        return out
    # Fallback: đi thẳng theo anchor nếu không bắt được card
    for a in soup.select("a[href*='/ban-'], a[href*='/nha-dat-'], a[href*='/can-ho-'], a[href*='/dat-']"):
        href = a.get("href","")
        if not href:
            continue
        link = href if href.startswith("http") else ("https://batdongsan.com.vn"+href)
        out.append({"link": link})
    return out


## 3) Selenium điều hướng

In [ ]:
def human_sleep(a=1.6, b=3.2):
    pause(a, b)

def human_scroll(driver, steps=8, bottom_pause=(0.8, 1.6)):
    h = driver.execute_script("return document.body.scrollHeight")
    y = 0
    for _ in range(steps):
        y += max(120, h // steps)
        driver.execute_script(f"window.scrollTo(0, {int(y)});")
        pause(0.4, 0.9)
    pause(*bottom_pause)

def click_consent_if_any(driver):
    texts = ["Đồng ý","Chấp nhận","Cho phép","Tôi hiểu","Accept","OK","Got it"]
    try:
        for b in driver.find_elements("css selector", "button, .btn, [role='button']"):
            t = (b.text or "").strip()
            if any(x.lower() in t.lower() for x in texts):
                try:
                    b.click()
                    pause(0.7, 1.0)
                    break
                except:
                    pass
    except:
        pass

def wait_for_cards(driver, timeout=25):
    t0 = time.time()
    while time.time() - t0 < timeout:
        n_cards = driver.execute_script("return document.querySelectorAll('div.re__card, div.property-item, div.js__card').length;")
        n_anchors = driver.execute_script("return document.querySelectorAll(\"a[href*='/ban-'], a[href*='/nha-dat-'], a[href*='/can-ho-'], a[href*='/dat-']\").length;")
        if (n_cards and n_cards > 0) or (n_anchors and n_anchors > 0):
            return True
        pause(0.9, 1.2)
    return False

def goto_page(driver, base_url: str, page_index: int):
    url = base_url if page_index <= 1 else f"{base_url}/p{page_index}"
    driver.get(url)

def click_next_page(driver, current_page_idx: int, base_url: str):
    tried = False
    try:
        for a in driver.find_elements("css selector", "a[rel='next'], a[aria-label*='Sau'], a[title*='Sau'], li a"):
            t = (a.get_attribute("aria-label") or "") + " " + (a.get_attribute("title") or "") + " " + (a.text or "")
            if any(x in t.lower() for x in ["sau","next","tiếp",">","»"]):
                a.click(); tried = True; break
    except: pass
    if not tried:
        goto_page(driver, base_url, current_page_idx + 1)

def crawl_list_links(n_pages: int, base_url: str, start_page: int):
    import undetected_chromedriver as uc
    from selenium.webdriver.chrome.options import Options
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu"); opts.add_argument("--no-sandbox"); opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("--window-size=1366,768"); opts.add_argument("--lang=vi-VN")
    # User-Agent phổ biến để giảm chặn
    opts.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = uc.Chrome(options=opts)
    results = []
    try:
        goto_page(driver, base_url, start_page)
        pause()
        click_consent_if_any(driver)
        human_scroll(driver, steps=10)
        wait_for_cards(driver, timeout=25)
        current_idx = start_page
        for i in range(n_pages):
            # Thử tối đa 2 lần nếu trang hiện tại chưa bắt được link
            attempts = 2
            page_items = []
            for _ in range(attempts):
                html = driver.page_source
                page_items = parse_list_page(html)
                if page_items:
                    break
                # scroll sâu thêm rồi đợi
                human_scroll(driver, steps=12)
                pause(1.0, 1.6)
                wait_for_cards(driver, timeout=10)
            seen = set(x["link"] for x in results)
            for it in page_items:
                if it["link"] not in seen:
                    results.append(it); seen.add(it["link"])
            print(f"[List Page {current_idx}] links (raw): {len(page_items)} | unique_total: {len(results)}")
            if len(page_items) == 0:
                print("[Hint] 0 link: trang có thể bị chặn/che overlay. Hãy thử HEADLESS=False (đã để mặc định) hoặc tăng DELAY_RANGE.")
            # Nếu còn trang phải xử lý thì mới chuyển trang
            if i < n_pages - 1:
                click_next_page(driver, current_page_idx=current_idx, base_url=base_url)
                current_idx += 1
                pause()
                click_consent_if_any(driver)
                human_scroll(driver, steps=8)
                wait_for_cards(driver, timeout=20)
    finally:
        driver.quit()
    return results

## 4) Regex-first trên section “Đặc điểm bất động sản”

In [ ]:

BAD_VALUES = {"tai ung dung","tải ứng dụng","xem thêm trên app","app","ứng dụng"}

def get_breadcrumb_type(soup: BeautifulSoup) -> str:
    for sel in ["nav[aria-label*='breadcrumb'] a", ".re__breadcrumb a", "ol.breadcrumb a"]:
        links = [text_clean(a.get_text()) for a in soup.select(sel) if text_clean(a.get_text())]
        if links:
            for txt in reversed(links):
                if "căn hộ" in txt.lower(): return txt
            return links[-1]
    a = soup.select_one("a.re__link-se")
    if a:
        txt = text_clean(a.get_text())
        if "căn hộ" in txt.lower(): return txt
    a2 = soup.select_one("nav a:last-child, ol.breadcrumb li:last-child a")
    return text_clean(a2.get_text()) if a2 else ""

def get_characteristics_text(soup: BeautifulSoup) -> str:
    # tìm section có 'Đặc điểm bất động sản', nếu không thấy dùng toàn trang
    for sec in soup.select("section, .re__section, .re__pr-specs"):
        head = sec.find(["h2","h3","h4"])
        if head and "đặc điểm" in text_clean(head.get_text()).lower():
            return sec.get_text("\n", strip=True)
    return soup.get_text("\n", strip=True)

def rex_search(patterns, text, flags=re.I):
    if isinstance(patterns, str): patterns = [patterns]
    for p in patterns:
        m = re.search(p, text, flags)
        if m: return m
    return None

def extract_fields_from_text(txt: str) -> dict:
    # Chuẩn hóa khoảng trắng nhưng GIỮ xuống dòng để bắt đầu dòng bằng nhãn
    t = re.sub(r"[ \t]+", " ", txt)

    def find_line_value(label_pattern: str) -> str:
        m = re.search(rf"^(?:{label_pattern})\s*[:\-]?\s*([^\n\r]+)", t, flags=re.I | re.M)
        return text_clean(m.group(1)) if m else ""

    # Giá (ưu tiên nhãn chính xác ở đầu dòng)
    gia = find_line_value(r"Khoảng\s*giá|Mức\s*giá|Giá")

    # Diện tích (dạng '70 m²' giữ nguyên số thập phân dùng dấu phẩy)
    m = re.search(r"^Diện\s*tích\s*[:\-]?\s*([0-9\.,]+ ?m²)", t, flags=re.I | re.M)
    dien_tich = text_clean(m.group(1)) if m else ""

    # Phòng ngủ
    pn_raw = find_line_value(r"Số\s*phòng\s*ngủ")

    # WC
    m_wc = re.search(r"^Số\s*phòng\s*tắm,\s*vệ\s*sinh\s*[:\-]?\s*([0-9]+)", t, flags=re.I | re.M)
    wc_raw = m_wc.group(1) if m_wc else find_line_value(r"(?:WC|Vệ\s*sinh|Toilet|Phòng\s*tắm)")

    # Nội thất
    noi_that_raw = find_line_value(r"Nội\s*thất")
    noi_that = ""
    if noi_that_raw:
        m2 = re.search(r"(đầy\s*đủ|full\s*nội\s*thất|cơ\s*bản|trống|không\s*nội\s*thất)", noi_that_raw, flags=re.I)
        noi_that = text_clean(m2.group(0)) if m2 else noi_that_raw

    # Pháp lý
    giay_to = find_line_value(r"(?:Pháp\s*lý|Giấy\s*tờ(?:\s*pháp\s*lý)?)")
    if any(bad in giay_to.lower() for bad in BAD_VALUES):
        giay_to = ""

    # Làm sạch 'phòng' khỏi số lượng
    def only_digits(s: str) -> str:
        m = re.search(r"\d+", s or "")
        return m.group(0) if m else (s or "").strip()

    so_pn = only_digits(pn_raw)
    so_wc = only_digits(wc_raw)

    # Loại bỏ giá trị giá không hợp lệ (ví dụ 'Biểu đồ giá')
    if gia and any(x in gia.lower() for x in {"biểu đồ giá", "liên hệ", "đang cập nhật"}):
        gia = ""

    return {
        "gia": gia,
        "dien_tich": dien_tich,
        "so_phong_ngu": so_pn,
        "so_phong_ve_sinh": so_wc,
        "tinh_trang_noi_that": noi_that,
        "giay_to_phap_ly": giay_to,
    }

def extract_ngay_dang(soup: BeautifulSoup) -> str:
    for node in soup.select("section, .re__pr-time, .re__pr-config, .re__pr-attribute, [class*='time'], [class*='date']"):
        txt = text_clean(node.get_text(" ", strip=True))
        if "ngày đăng" in txt.lower():
            m = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", txt)
            if m: return m.group(1)
    m = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", soup.get_text(" ", strip=True))
    return m.group(1) if m else ""

def extract_detail_v8(html: str, link: str) -> Dict[str, str]:
    soup = BeautifulSoup(html, "lxml")
    sec_text = get_characteristics_text(soup)
    fields = extract_fields_from_text(sec_text)
    return {
        "ngay_dang": extract_ngay_dang(soup),
        "loai_hinh": get_breadcrumb_type(soup),
        "dien_tich": fields["dien_tich"],
        "gia": fields["gia"],
        "giay_to_phap_ly": fields["giay_to_phap_ly"],
        "so_phong_ngu": fields["so_phong_ngu"],
        "so_phong_ve_sinh": fields["so_phong_ve_sinh"],
        "tinh_trang_noi_that": fields["tinh_trang_noi_that"],
        "link": link,
    }

## 5) Crawl → chi tiết → 9 cột

In [ ]:
def crawl_v8(n_pages: int, base_url: str, start_page: int) -> pd.DataFrame:
    links = crawl_list_links(n_pages, base_url, start_page)
    if not links: return pd.DataFrame(columns=COLS9)

    if VERBOSE_DETAIL:
        print(f"[Detail] Tổng link sẽ crawl: {len(links)}")

    import undetected_chromedriver as uc
    from selenium.webdriver.chrome.options import Options
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu"); opts.add_argument("--no-sandbox"); opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1366,768"); opts.add_argument("--lang=vi-VN")
    driver = uc.Chrome(options=opts)

    rows = []
    try:
        total = len(links)
        for idx, it in enumerate(links, start=1):
            url = it["link"]
            success = False
            for attempt in range(1, 3):  # 2 attempts
                try:
                    driver.get(url)
                    pause(1.8, 2.8)
                    html = driver.page_source
                    rows.append(extract_detail_v8(html, link=url))
                    if VERBOSE_DETAIL:
                        print(f"[Detail {idx}/{total}] OK")
                    pause(0.8, 1.4)
                    success = True
                    break
                except Exception as e:
                    if VERBOSE_DETAIL:
                        print(f"[Detail {idx}/{total}] Retry {attempt} error: {e}")
                    pause(1.2, 2.0)
            if not success:
                miss = {k:"" for k in COLS9}; miss["link"] = url; rows.append(miss)
                if VERBOSE_DETAIL:
                    print(f"[Detail {idx}/{total}] FAIL -> added empty row")
    finally:
        driver.quit()

    return pd.DataFrame(rows, columns=COLS9)

## 6) Append theo link (ưu tiên dữ liệu mới không rỗng)

In [ ]:

def smart_append_by_link(old_df: pd.DataFrame, new_df: pd.DataFrame) -> pd.DataFrame:
    for c in COLS9:
        if c not in old_df.columns: old_df[c] = ""
        if c not in new_df.columns: new_df[c] = ""
    old_df = old_df[COLS9].drop_duplicates(subset=["link"], keep="first")
    new_df = new_df[COLS9].drop_duplicates(subset=["link"], keep="first")
    both = pd.concat([old_df, new_df], ignore_index=True).sort_values("link")

    def pick_nonempty(series):
        for v in series[::-1]:
            if pd.notna(v) and str(v).strip() != "":
                return v
        return series.iloc[-1]

    merged = both.groupby("link", as_index=False).agg({c: pick_nonempty for c in COLS9})
    return merged[COLS9]


## 7) RUN ALL

In [ ]:

print("BASE_URL:", BASE_URL, "| START_PAGE:", START_PAGE, "| N_PAGES:", N_PAGES)
df_new = crawl_v8(N_PAGES, BASE_URL, START_PAGE)
print("Số dòng mới:", len(df_new))

csv_path = Path(OUTPUT_CSV)
if APPEND_MODE and csv_path.exists():
    old_df = pd.read_csv(csv_path)
    final_df = smart_append_by_link(old_df, df_new)
else:
    final_df = df_new

final_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Đã lưu CSV: {OUTPUT_CSV} (tổng {len(final_df)} dòng)")
display(final_df.head(20))
